In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from lib.parse_utils import *
from lib.ocr_utils import *
from lib.pipeline_runner import *

company_id = Please check it in the DB

In [ ]:
company_id = 1
country = "USA"
processed_date = pd.Timestamp.now().strftime("%Y-%m-%d")

<a id="rg"></a>

---

<h3>Reading file path</h3>

---

In [ ]:
# Define project-relative base directory (portable)
base_dir = os.path.join(os.getcwd(), "data")

# Define the search pattern for all company folders
path_pattern = os.path.join(base_dir, "company-*/raw/**/*")

# Collect all PDF paths recursively, ignoring ".ini" files
path = [
    p.replace("\\", "/")
    for p in glob(path_pattern, recursive=True)
    if ".ini" not in p
]

# Show the first few paths
path[:]

In [ ]:
df = run_extraction_pipeline(path, company_id=company_id, country=country, processed_date=processed_date)

In [ ]:
df.head(20)

---
Connecting to the database (Postgres)

In [ ]:
from db_connection import postgres
db = postgres()

---
Uploading the files to the database (Postgres)

In [ ]:
if df.empty:
    print("✅ No new invoices to upload — all records already exist.")
else:
    print(f"🚀 Uploading {len(df)} new invoices to the database...")

    inserted_count = db.insert_dataframe(df, "invoices")

    if inserted_count > 0:
        print(f"✅ Successfully uploaded {inserted_count} new invoices to PostgreSQL.")
    elif inserted_count == 0:
        print("❌ No new invoices were uploaded — all were duplicates.")
    else:
        print("❌ Upload failed — check error log above.")


---